## Part 1: "The Eye" - AI-First Rush Detection
This module uses OpenCV to detect and count rectangular order tickets on a kitchen rail. In a production environment, this runs locally on a low-cost camera, bypassing human input bias.

In [ ]:
def count_tickets_in_image(image_path=None):
    """
    Simulates ticket counting. 
    If no image is provided, it generates a synthetic image of a ticket rail for demonstration.
    """
    if image_path:
        img = cv2.imread(image_path)
    else:
        # Generate a synthetic dark background with white "tickets" for the simulation
        img = np.zeros((400, 800, 3), dtype=np.uint8)
        num_mock_tickets = random.randint(3, 12)
        for i in range(num_mock_tickets):
            x = 50 + (i * 60)
            cv2.rectangle(img, (x, 100), (x+40, 250), (255, 255, 255), -1)
            
    # CV Pipeline
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    _, thresh = cv2.threshold(blurred, 200, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    ticket_count = 0
    output_img = img.copy()
    
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area > 1000: # Filter out noise
            x, y, w, h = cv2.boundingRect(cnt)
            cv2.rectangle(output_img, (x, y), (x+w, y+h), (0, 255, 0), 3)
            ticket_count += 1
            
    # Visualize the result inline for the judges
    plt.figure(figsize=(10, 4))
    plt.imshow(cv2.cvtColor(output_img, cv2.COLOR_BGR2RGB))
    plt.title(f"Detected Live Kitchen Rush: {ticket_count} Tickets")
    plt.axis('off')
    plt.show()
    
    return ticket_count

# Run the simulation
current_live_tickets = count_tickets_in_image()
print(f"Objective Signal Extracted: {current_live_tickets} active orders.")

## Part 2: KPT Adjustment Engine & Synthetic Dataset
How does this data actually improve ETA? 

Standard KPT models rely on fixed historical averages and biased merchant "Food Ready" marks. KitchenSync dynamically alters the Base KPT using:
1. **CV Ticket Count (The Eye):** +1.5 minutes per active ticket over the baseline capacity.
2. **Merchant Pulse (The UI):** 🔴 Swamped (+10 mins), 🟡 Steady (0 mins), 🟢 Quiet (-3 mins).

In [ ]:
def generate_kitchen_dataset(num_orders=100):
    data = []
    base_time = datetime.now()
    
    for i in range(num_orders):
        # 1. Standard Zomato inputs
        order_id = f"ORD_{1000+i}"
        base_kpt = random.randint(12, 18) # Standard algorithmic KPT in minutes
        
        # 2. KitchenSync enriched signals
        # Simulating dine-in rush that Zomato normally can't see
        cv_ticket_count = random.randint(2, 20) 
        
        # Simulating frictionless merchant UI input
        if cv_ticket_count > 15:
            pulse_status = "Swamped"
        elif cv_ticket_count < 6:
            pulse_status = "Quiet"
        else:
            pulse_status = "Steady"
            
        # 3. The KitchenSync Adjustment Math
        # Assume kitchen handles 5 tickets normally. Every ticket above 5 adds 1.5 mins delay.
        cv_delay_penalty = max(0, (cv_ticket_count - 5) * 1.5)
        
        pulse_modifier = 0
        if pulse_status == "Swamped": pulse_modifier = 10
        elif pulse_status == "Quiet": pulse_modifier = -3
        
        # Calculate new dynamic KPT
        kitchensync_kpt = base_kpt + cv_delay_penalty + pulse_modifier
        
        data.append({
            "Order_ID": order_id,
            "Base_Algorithmic_KPT": base_kpt,
            "CV_Ticket_Count": cv_ticket_count,
            "Merchant_Pulse_Status": pulse_status,
            "KitchenSync_Adjusted_KPT": round(kitchensync_kpt, 1)
        })
        
    return pd.DataFrame(data)

# Generate and view the dataset
df = generate_kitchen_dataset(15)
display(df.head(10))

In [ ]:
# Visualize the difference between Standard KPT and KitchenSync KPT
plt.figure(figsize=(12, 6))
plt.plot(df.index, df['Base_Algorithmic_KPT'], label='Standard KPT (Blind to Live Rush)', marker='o', linestyle='--')
plt.plot(df.index, df['KitchenSync_Adjusted_KPT'], label='KitchenSync KPT (Enriched Signals)', marker='x', linewidth=2)

plt.fill_between(df.index, df['Base_Algorithmic_KPT'], df['KitchenSync_Adjusted_KPT'], color='gray', alpha=0.2, label='Hidden Wait Time Prevented')

plt.title('Impact of Signal Enrichment on KPT Prediction')
plt.xlabel('Order Sequence')
plt.ylabel('Predicted Prep Time (Minutes)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import random

# Set random seed for reproducibility
np.random.seed(42)
random.seed(42)

# KitchenSync: KPT Optimization Simulation
**Focus:** Enhancing Kitchen Prep Time (KPT) signals via hardware-assisted CV and frictionless merchant UI.

This notebook demonstrates:
1. **The Eye:** A Computer Vision prototype to objectively count physical order tickets and measure live kitchen rush.
2. **The Pulse & KPT Engine:** Generating a synthetic dataset to simulate how CV ticket counts and 1-tap merchant inputs adjust the baseline KPT predictions.